[View in Colaboratory](https://colab.research.google.com/github/Simolx/colaboratory/blob/nmt/nmt/nmt.ipynb)

# NMT

## 1.   mount google drive, prepare data



In [2]:
from google.colab import drive
drive.mount('/mt')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /mt


In [9]:
! ls /mt/My\ Drive/mt/icytranslate_offline/data

c.clean.ja  c.clean.zh


In [0]:
! mkdir data

In [0]:
! ! cp /mt/My\ Drive/mt/icytranslate_offline/data/* ./data/

In [13]:
! ls data

c.clean.ja  c.clean.zh
